In [1]:
import pandas as pd

## Data

In [2]:
import json

def read_json(file):
    with open(file) as f:
        return json.load(f)


In [3]:
# df = pd.read_csv('eval_scenarios/reception_agent_adapt.csv',delimiter=";")
df = pd.read_csv('outputs/try_agent_reception_20241011-141555.csv')
conditions = read_json('eval_scenarios/conditions.json')
goals = read_json('eval_scenarios/goals.json')

In [4]:
df.columns

Index(['num_condition', 'type', 'goals', 'GT_subgoals_disfavour',
       'GT_subgoals_favour', 'agent', 'subgoals_disfavour', 'subgoals_favour'],
      dtype='object')

In [5]:
df['condition'] = df['num_condition'].apply(lambda x: conditions[str(x)]['condition'])
df['disfavour_condition'] = df['num_condition'].apply(lambda x: conditions[str(x)]['disfavour'])

In [6]:
# Función para seleccionar los subgoals correctos a partir de múltiples goals en una fila
def get_subgoals(row, conditions):
    condition = row['condition']
    goal_ids = eval(row['goals'])  # Suponemos que los ids de los goals están separados por comas
    subgoals_list = []
    # Iterar sobre todos los goal_ids y obtener los subgoals correspondientes
    for goal_id in goal_ids:
        goal_id = goal_id.strip()  # Eliminar espacios en blanco
        if 'human' in condition:
            subgoals_list.extend(goals[goal_id]['subgoals_human'])
        elif 'robot' in condition:
            subgoals_list.extend(goals[goal_id]['subgoals_robot'])
    
    return subgoals_list

# Crear la nueva columna con los subgoals concatenados en una lista
df['subgoals'] = df.apply(lambda row: get_subgoals(row, conditions), axis=1)

## Execute LLM

In [7]:
import openai
import os

from llm_env_to_goal import LLMEnvToGoalReasoner
from llm_agent_to_action import LLMAgentToActionAllocationReasoner

from dotenv import load_dotenv

In [8]:
openai.api_key = os.getenv("OPENAI_API_KEY")
llm_agent_to_action = LLMAgentToActionAllocationReasoner("prompts")

def apply_llm_action_costs(row):
    # Llamar a la función con los subgoals y condition de la fila actual
    agent, subgoals_disfavour, subgoals_favour, pddl_costs = llm_agent_to_action.llm_pddl_action_costs_for_agent_condition(
        str(row['subgoals']), row['condition']
    )
    
    # Devolver los valores en un diccionario para agregarlos como nuevas columnas
    return pd.Series({
        'agent': agent,
        'subgoals_disfavour': subgoals_disfavour,
        'subgoals_favour': subgoals_favour,
        'pddl_costs': pddl_costs
    })


In [11]:
# Aplicar la función a cada fila y asignar las nuevas columnas al DataFrame
df[['agent', 'subgoals_disfavour', 'subgoals_favour']] = df.apply(lambda row: apply_llm_action_costs(row)[:-1], axis=1)

El agente deberia desfavorecer estas tareas:
Ninguna

El agente deberia favorecer estas tareas:
Ninguna
[]
['']
['']
El agente deberia desfavorecer estas tareas:
Ninguna

El agente deberia favorecer estas tareas:
['carta colocada sobre', 'sello puesto sobre', 'sobre enviado correus', 'carrito recogido loc3', 'paquete recogido loc2_entradaFME', 'carrito dejado loc3', 'registros colocados caja', 'caja llevada almacen']
[]
['', "['carta colocada sobre', 'sello puesto sobre', 'sobre enviado correus', 'carrito recogido loc3', 'paquete recogido loc2_entradaFME', 'carrito dejado loc3', 'registros colocados caja', 'caja llevada almacen']"]
['']
["['carta colocada sobre'", "'sello puesto sobre'", "'sobre enviado correus'", "'carrito recogido loc3'", "'paquete recogido loc2_entradaFME'", "'carrito dejado loc3'", "'registros colocados caja'", "'caja llevada almacen']"]
{'attribute': "'sello puesto sobre'_cost", 'agent': 'human', 'obj': "['carta colocada sobre'", 'loc': "'sobre enviado correus'", 

TypeError: cannot unpack non-iterable bool object

In [10]:
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
df.drop(columns=['condition','subgoals']).to_csv(f"outputs/try_agent_reception_{timestr}.csv",index=False)

## Add GT and calculate

In [11]:
df['GT_subgoals_disfavour'] = df.apply(lambda row: list(set(row['subgoals']).intersection(row['disfavour_condition'])), axis=1)

In [12]:
df.drop(columns=['condition','subgoals','disfavour_condition'],inplace=True)

In [13]:
df['GT_subgoals_disfavour'].values[2]

[]

In [14]:
df['subgoals_disfavour'] = df['subgoals_disfavour'].apply(lambda x: eval(x))

TypeError: eval() arg 1 must be a string, bytes or code object

In [56]:

# Función para calcular Completeness y Correctness
def calculate_completeness_correctness(row):
    gt_set = set(row['GT_subgoals_disfavour'])
    subgoal_set = set(row['subgoals_disfavour'])
    
    if len(gt_set) == 0:
        completeness = 1.0 if len(subgoal_set) == 0 else 0.0
    else:
        completeness = len(subgoal_set & gt_set) / len(gt_set)
    
    if len(subgoal_set) == 0:
        correctness = 1.0 if len(gt_set) == 0 else 0.0
    else:
        correctness = len(subgoal_set & gt_set) / len(subgoal_set)
    
    return pd.Series([completeness, correctness])

# Aplicar la función al DataFrame
df[['Completeness', 'Correctness']] = df.apply(calculate_completeness_correctness, axis=1)


In [58]:
df.describe()

,num_condition,Completeness,Correctness
count,27.000000,27.000000,27.000000
mean,5.000000,0.462963,0.481481
std,2.631174,0.498573,0.509175
min,1.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000
50%,5.000000,0.000000,0.000000
75%,7.000000,1.000000,1.000000
max,9.000000,1.000000,1.000000


In [60]:
df[(df['Completeness']==1)&(df['Correctness']==1)]

,num_condition,type,goals,GT_subgoals_disfavour,GT_subgoals_favour,agent,subgoals_disfavour,subgoals_favour,Completeness,Correctness
0,1,1 - Robot negative,"['1','3','5']",[],none,robot,[],"[""['letter placed envelope', 'stamp put envelo...",1.0,1.0
2,3,1 - Robot negative,"['1','3','5']",[package picked_up loc2_entradaFME],none,robot,[package picked_up loc2_entradaFME],[],1.0,1.0
5,6,1 - Robot negative,"['1','3','5']",[],none,human,[],[],1.0,1.0
7,8,1 - Robot negative,"['1','3','5']",[],none,human,[],"[""['letter placed envelope', 'stamp put envelo...",1.0,1.0
9,1,1 - Robot negative,"['1','2','7']",[cutter used_to_open package],none,robot,[cutter used_to_open package],[],1.0,1.0
14,6,1 - Robot negative,"['1','2','7']",[material organized loc1],none,human,[material organized loc1],[],1.0,1.0
18,1,1 - Robot negative,"['2','3','4','6']",[],none,robot,[],"[""['diplomas stamped loc1', 'package picked_up...",1.0,1.0
19,2,1 - Robot negative,"['2','3','4','6']",[visitor attended loc4],none,robot,[visitor attended loc4],[],1.0,1.0
20,3,1 - Robot negative,"['2','3','4','6']",[package picked_up loc2_entradaFME],none,robot,[package picked_up loc2_entradaFME],[],1.0,1.0
21,4,1 - Robot negative,"['2','3','4','6']",[diplomas stamped loc1],none,human,[diplomas stamped loc1],"['cart picked_up loc3', 'package picked_up loc...",1.0,1.0


In [61]:
df

,num_condition,type,goals,GT_subgoals_disfavour,GT_subgoals_favour,agent,subgoals_disfavour,subgoals_favour,Completeness,Correctness
0,1,1 - Robot negative,"['1','3','5']",[],none,robot,[],"[""['letter placed envelope', 'stamp put envelo...",1.0,1.0
1,2,1 - Robot negative,"['1','3','5']",[],none,robot,"[cart picked_up loc3, package picked_up loc2_e...","['letter placed envelope', 'stamp put envelope...",0.0,0.0
2,3,1 - Robot negative,"['1','3','5']",[package picked_up loc2_entradaFME],none,robot,[package picked_up loc2_entradaFME],[],1.0,1.0
3,4,1 - Robot negative,"['1','3','5']",[],none,human,"[letter placed envelope, stamp put envelope]","['cart picked_up loc3', 'package picked_up loc...",0.0,0.0
4,5,1 - Robot negative,"['1','3','5']","[envelope sent correus, package picked_up loc2...",none,robot,[],[],0.0,0.0
5,6,1 - Robot negative,"['1','3','5']",[],none,human,[],[],1.0,1.0
6,7,1 - Robot negative,"['1','3','5']",[envelope sent correus],none,robot,[],[],0.0,0.0
7,8,1 - Robot negative,"['1','3','5']",[],none,human,[],"[""['letter placed envelope', 'stamp put envelo...",1.0,1.0
8,9,1 - Robot negative,"['1','3','5']","[stamp put envelope, letter placed envelope]",none,human,[],[],0.0,0.0
9,1,1 - Robot negative,"['1','2','7']",[cutter used_to_open package],none,robot,[cutter used_to_open package],[],1.0,1.0
